## Tekstin Luokittelu

## Datan kuvaus
Datasetti pitää sisällään yli 100 000 elokuva-arvostelua, jotka on jaettu positiiivisiksi ja negatiivisiksi arvosteluiksi. Data on tekstimuodossa, ja siitä pitää vain karsia muutamia merkkejä.

In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


## Datan esikäsittely
Ensimmäinen vaihe on hakea datasetti ja jakaa se `train` ja `val` setteihin. Tiedostot myös sekoitetaan käyttämällä tiettyä seediä, joka on tässä tapauksessa `1337`.
Poistimme myös manuaalisesti `unsup`-nimisen kansion, joka piti sisällään elokuva-arvosteluita, joita ei oltu suoraan luokiteltu positiivisiksi tai negatiivisiksi. 

In [ ]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val" 
train_dir = base_dir / "train" 
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)              
    num_val_samples = int(0.2 * len(files))         
    val_files = files[-num_val_samples:]            
    for fname in val_files:                         
        shutil.move(train_dir / category / fname, val_dir / category / fname)     

Seuraavassa koodisolussa haemme jaetut datasetit omiin muuttujiinsa.

In [7]:
from tensorflow import keras
batch_size = 32 
  
train_ds = keras.utils.text_dataset_from_directory(     
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


## Mallinnus

Seuraavaksi määritellään 'TransformerEncoder' luokka jonka, tarkoituksena on antaa sanoille konteksti riippuvainen merkitys (Sana saa perinteisen embedded vektorin sijaan konteksti riippuvaisen vektorin). Tämä auttaa mallia ymmärtämään sanojen järjestystä sekä kontekstia kun tehdään tekstin luokittelua. Meidän encoder luokka sisältää call() nimisen metodin joka suoriutuu kun luokasta tehdään objekti.

In [8]:
import tensorflow as tf
from tensorflow.keras import layers
  
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
  
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

`PositionalEmbedding` luokan ideana on toimia `Embedding`-kerroksena, joka enkoodaa tokenit ja niiden sijainnit sanassa. Sijanti on tärkeää tietää lauseiden luonnissa, koska sanojen sijainnit lauseessa voi vaikuttaa suuresti sanan tarkoitukseen.

In [5]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
  
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
 
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
 
    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

Seuraavan solun tarkoitus on vektorisoida jokainen datasetti numeroiksi, jotta voimme käyttää niitä mallin kouluttamiseen.

In [9]:
from tensorflow.keras import layers

text_only_train_ds = train_ds.map(lambda x, y: x)
  
max_length = 600 
max_tokens = 20000 
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,     
)
text_vectorization.adapt(text_only_train_ds)
 
int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

Luomme seuraavaksi mallin, johon asetamme `input`-kerroksen jälkeen aiemmin luomat `PositionalEmbedding` ja `TransformerEncoder` kerrokset. Näitä kerroksia seuraa yksi `GlobalMaxPooling1D`-kerros, joka karsii korkeimman arvon pois ja tekee shapesta yksiulotteisen. Lisäämme yhden `Dropout` kerroksen, välttääkseen ylioppimista, mutta muuten kyseinen malli on hyvin suoraviivainen.

Tallennamme mallin `full_transformer_encoder.keras` tiedostoon, jotta sitä voi käyttää helpommin koulutuksen jälkeen. Koulutamme mallia `20` epochia.

In [10]:
vocab_size = 20000 
sequence_length = 600 
embed_dim = 256 
num_heads = 2 
dense_dim = 32 
  
inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()
  
callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras", save_best_only=True)
] 
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)
model = keras.models.load_model(
    "full_transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder, "PositionalEmbedding": PositionalEmbedding}) 
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding_1 (Pos  (None, None, 256)        5273600   
 itionalEmbedding)                                               
                                                                 
 transformer_encoder_1 (Tran  (None, None, 256)        543776    
 sformerEncoder)                                                 
                                                                 
 global_max_pooling1d_1 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           

## Arviointi
Tuloksista päätellen malli alkaa ylioppimaan jo ensimmäisen epochin jälkeen, ja saamme parhaan tuloksen samaisella epochilla. Tästä huolimatta saavutamme silti korkean tarkkuus arvon `88%`.

## Käyttöönotto
Tätä mallia voisi hyödyntää mihin tahansa suodatukseen. Hyviä esimerkkejä olisi sähköpostin suodatus tai jonkun yrityksen käyttäjäpalautteiden suodatus. Näihin tarkoituksiin pitäisi olla tietenkin omanlaiset datasetit, mutta malli voisi olla samanlainen.